In [1]:
print("OK")

OK


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

/home/rudra/anaconda3/envs/mChatbot/lib/python3.8/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [33]:
PINECONE_API_KEY = "bc70c04f-1d63-4176-89f7-d938e18a0cb1"
PINECONE_API_ENV = "gcp-starter"

In [4]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [6]:
extracted_data = load_pdf("Data/")

In [7]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [9]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [10]:
embeddings = download_hugging_face_embeddings()


In [11]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [12]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [13]:
query_result

[-0.03447727486491203,
 0.031023237854242325,
 0.006734995637089014,
 0.026108987629413605,
 -0.039362020790576935,
 -0.1603025197982788,
 0.06692394614219666,
 -0.006441484205424786,
 -0.04745049774646759,
 0.014758791774511337,
 0.07087524980306625,
 0.05552758276462555,
 0.01919332891702652,
 -0.026251375675201416,
 -0.010109515860676765,
 -0.02694043703377247,
 0.0223073810338974,
 -0.022226618602871895,
 -0.1496926099061966,
 -0.017493048682808876,
 0.0076762172393500805,
 0.054352305829524994,
 0.003254448063671589,
 0.03172590956091881,
 -0.0846213772892952,
 -0.02940598502755165,
 0.0515955314040184,
 0.0481240339577198,
 -0.0033148115035146475,
 -0.058279164135456085,
 0.04196930676698685,
 0.022210676223039627,
 0.1281888335943222,
 -0.02233891747891903,
 -0.011656291782855988,
 0.06292839348316193,
 -0.03287632763385773,
 -0.09122605621814728,
 -0.03117537684738636,
 0.05269954726099968,
 0.047034818679094315,
 -0.0842030867934227,
 -0.030056215822696686,
 -0.020744841545820

In [25]:
from pinecone import Pinecone

pc = Pinecone(api_key="bc70c04f-1d63-4176-89f7-d938e18a0cb1")
index = pc.Index("m-chatbot")

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec

# Assuming you have your API key and environment variables defined somewhere
# PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
# PINECONE_API_ENV = os.environ.get("PINECONE_API_ENV")

# Create an instance of Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Now you can perform operations using this instance
index_name = "m-chatbot"

# Assuming text_chunks and embeddings are defined somewhere in your code
docsearch = pc.create_index(
    name=index_name,
    texts=[t.page_content for t in text_chunks],  # Assuming page_content is the text you want to index
    embeddings=embeddings,
    metric='euclidean',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-west-2'
    )
)

# docsearch=Pinecone.from_texts(
#     [t.page_content for t in text_chunks],
#      embeddings, 
#      index_name=index_name
#     )


In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec

# Assuming you have your API key and environment variables defined somewhere
# PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
# PINECONE_API_ENV = os.environ.get("PINECONE_API_ENV")

# Create an instance of Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Now you can perform operations using this instance
index_name = "m-chatbot"

# Assuming text_chunks and embeddings are defined somewhere in your code
docsearch = pc.create_index(
    name=index_name,
    dimension=384,  # Assuming the dimensionality of your embeddings is 384
    metric='euclidean',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-west-2'
    )
)

# Assuming text_chunks and embeddings are defined somewhere in your code
# You can also add data directly to the index if needed
for t in text_chunks:
    docsearch.upsert(item=t.page_content, vector=embeddings)  

# If you want to store additional metadata, you can modify the above loop accordingly

# Alternatively, if you want to use Pinecone.from_texts()
# docsearch = Pinecone.from_texts(
#     [t.page_content for t in text_chunks],
#     index_name=index_name
# )


In [28]:
#Initializing the Pinecone
pinecone.init(api_key= PINECONE_API_KEY,
              environment= PINECONE_API_ENV)

index_name="m-chatbot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )

